# 키 확인

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))

True

# 실행확인

In [2]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import schedule
import time

flag=0
load_dotenv()
key=os.getenv('KEY')
# def crawl_job():
url = "https://data-dbg.krx.co.kr/svc/apis/idx/krx_dd_trd"
params = {
    "AUTH_KEY": key,
    "basDd": 20251229,   # 기준일자 (당일 데이터 없는지 확인)
    'IDX_NM' : 'KRX 300',
    "REQ_TYPE": "json"
}

response = requests.get(url, params=params) 
data = response.json()

# JSON 파싱
if "OutBlock_1" in data and data["OutBlock_1"]:
    df = pd.json_normalize(data["OutBlock_1"])
    df = df[["BAS_DD", "IDX_NM", "CLSPRC_IDX", "ACC_TRDVOL"]]
    print(df)
else:
    print("데이터 없음")
    
# schedule.every().day.at("16:00").do(crawl_job)

# while flag==0:
#     schedule.run_pending()
#     time.sleep(30)

데이터 없음


# 실행코드

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
load_dotenv()
key=os.getenv('KEY')

url = "https://data-dbg.krx.co.kr/svc/apis/idx/krx_dd_trd"

start_date = datetime(2025, 9, 30)   # 시작일
end_date   = datetime.today()  # 종료일

all_data = []
not_in_data = []

while start_date <= end_date:
    date_str = start_date.strftime("%Y%m%d")  # 'YYYYMMDD' 형식
    params = {
        "AUTH_KEY": key,
        "basDd": date_str,
        "REQ_TYPE": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()

    if "OutBlock_1" in data and data["OutBlock_1"]:
        # 원하는 IDX_NM만 필터링
        for row in data["OutBlock_1"]:
            if row["IDX_NM"] in ["KRX 건설", "KRX 자동차", "KRX 헬스케어"]:
                all_data.append(row)
    
    # 휴무일을 확인할 필요가 있는 경우
    #else:
        ## 휴무일, 장 중에는 데이터가 없음
        #print(f"{date_str} 데이터 없음")
        ## 날짜 저장 리스트
        #not_in_data.append(date_str)

    start_date += timedelta(days=1)

# DataFrame으로 변환 후 필요한 열만 추출
df = pd.DataFrame(all_data)[["BAS_DD", "IDX_NM", "CLSPRC_IDX", "ACC_TRDVOL"]]
df.columns = ["날짜", "종목명", "종가(백만원)", "거래량(천주)"]

# 증감치열 추가
df.sort_values(by=['날짜','종목명'],ascending=[True,True], inplace=True)
df.reset_index(drop=True, inplace=True)

check_data = df.iloc[:,-2].copy()
KRX_items = set(df.iloc[:,1])

data_count = len(check_data)
KRX_count = len(KRX_items)
date_count = int(data_count/KRX_count)

up_down_list = []
# 전체데이터에 날짜당 아이템 갯수를 나누어 묶음들을 구함
for date_idx in range(date_count):
    # 묶음 내부의 값들의 증감치 추가
    # 첫 묶음은 비교대상이 없기에 '시작일'로 지정
    if date_idx == 0:
        for start_idx in range(KRX_count):
            up_down_list.append(0)
    else:
        # 현재 묶음과 이전 묶음의 각각의 아이템의 크기를 비교 
        for udw_idx in range(KRX_count):
            # 전일대비 증감치를 숫자로 표기, 소수점 둘째까지 반올림
            up_down_list.append(round(float(check_data[date_idx*3 + udw_idx])-float(check_data[(date_idx-1)*3 + udw_idx])
                                            ,2))

df['전일대비'] = up_down_list
df.to_csv('data/주가지수크롤링.csv')
print('작업완료')

c:\Users\Admin\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.5.0) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


작업완료


# 확인코드

In [3]:
df.tail(6)

,날짜,종목명,종가(백만원),거래량(천주),전일대비
168,20251224,KRX 건설,833.39,52730142,1.16
169,20251224,KRX 자동차,2502.26,8634285,5.89
170,20251224,KRX 헬스케어,4737.30,22673851,-38.23
171,20251226,KRX 건설,821.56,33118642,-11.83
172,20251226,KRX 자동차,2469.70,8059058,-32.56
173,20251226,KRX 헬스케어,4770.53,24416928,33.23
